# GEL 

In [ ]:
from IPython.display import Image
Image(filename='GELreal.png', width=800)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ngsolve import *
from netgen.csg import *
from netgen.meshing import MeshingParameters, meshsize
from ngsolve.webgui import Draw

In [2]:
class gel_bonded:
    def __init__(self):
        self.phi0 = 0.2035
        self.entropic_unit = 136.6
        self.G = 0.13
        self.gamma = self.G/self.entropic_unit # 0.0009516837481698391 self.compute_gamma( lamb =1.4874):
        J_iso = 3.2908
        self.lambda_iso = 1.4874 # =J**(1/3)     
        self.lambda_target = 2.1028   
        self.chi =  0.3704756498924026# compute_chi(phi0=0.2035, gamma =0.0009516837481698391, J=J_iso)
        self.geometry_bonded()

    def geometry_bonded(self):
        self.L = 90
        self.d = 1.62
        self.L3 = 15
        left  = Plane (Pnt(-0.5*self.L,0,0), Vec(-1,0,0) )
        right = Plane (Pnt(0.5*self.L,0,0), Vec( 1,0,0) )
        bot   = Plane (Pnt(0,0,0), Vec(0,-1,0) ).bc('bottom')
        top   = Plane (Pnt(0,self.d,0), Vec(0,1,0) )
        back  = Plane (Pnt(0,0,-0.5*self.L3), Vec(0,0, -1) )
        front = Plane (Pnt(0,0,0.5*self.L3), Vec(0,0,1) )
        brick = left * right * front * back * bot * top
        self.geo = CSGeometry()
        self.geo.Add (brick)
        
    def compute_chi( self,phi0, gamma, J):
        phi = phi0/J
        return -phi**(-2) * ((1-1/1000) * phi + log(1-phi) + gamma*J**(-1/3) )
    # compute gamma using free swelling model
    def compute_gamma(self, lamb):
        return -self.dH(lamb**3)*lamb
    # compute gamma using uniaxial approximation
    def gammafun(self,lamb):
        return -self.dH(lamb)/lamb
    def phi(self, J):
        return self.phi0/J

    def H(self, J):
        return (J - self.phi0)*log(1-self.phi(J))  + self.phi0 * self.chi*(1-self.phi(J)) + self.phi0/1000*log(self.phi(J))

    def dH(self, J):
        return (1-1/1000) * self.phi(J) + np.log(1-self.phi(J)) + self.chi * self.phi(J)**2

    # energy density
    def W(self, F):
        J = Det(F)
        C = F.trans* F
        return 0.5*self.gamma * Trace(C) + self.H(J)

In [3]:
class Solve_gel_bonded:
    def __init__(self, gel, maxh=5, corner_refinement=False):
        self.gel = gel
        if corner_refinement:
            mp = self.corner_refinement(maxh=maxh)
            self.mesh = Mesh(gel.geo.GenerateMesh(mp=mp))
        else:
            self.mesh = Mesh(gel.geo.GenerateMesh(maxh=maxh))
        
    def corner_refinement(self, maxh):
        mp = MeshingParameters(maxh=maxh)
        hc = maxh/10
        mp.RestrictH(x = -0.5*self.gel.L, y = 0 , z = -0.5*self.gel.L3 , h = hc)
        mp.RestrictH(x = -0.5*self.gel.L, y = 0 , z = 0.5*self.gel.L3 ,  h = hc)
        mp.RestrictH(x = 0.5*self.gel.L, y = 0 , z = -0.5*self.gel.L3 ,  h = hc)
        mp.RestrictH(x = 0.5*self.gel.L, y = 0 , z = 0.5*self.gel.L3 ,  h = hc)

        mp.RestrictH(x = -0.5*self.gel.L, y = self.gel.d , z = -0.5*self.gel.L3 ,  h = hc)
        mp.RestrictH(x = -0.5*self.gel.L, y = self.gel.d , z = 0.5*self.gel.L3 ,  h = hc)
        mp.RestrictH(x = 0.5*self.gel.L, y = self.gel.d , z = -0.5*self.gel.L3 ,  h = hc)
        mp.RestrictH(x = 0.5*self.gel.L, y = self.gel.d , z = 0.5*self.gel.L3 , h = hc)
        return mp

    def Solve_direct(self, gfu=None):
        self.Space()
        if gfu is None:
            self.gfu = GridFunction(self.fes)
            self.gfu.vec[:] = 0
        self.model()
        self.gfu, _,_ = SolveNonlinearMinProblem(a= self.a, gfu = self.gfu, FreeDofs =self.fes.FreeDofs(), maxits=maxits)

    def Solve_incremental_softening(self):
        self.Space()
        self.gfu = GridFunction(self.fes)
        self.gfu.vec[:] = 0
        
        gamma_end = self.gel.gamma
        gamma0 = self.gel.gammafun(1.1)
        self.gel.gamma = Parameter(gamma0)
        self.model()
        lambda_list = np.linspace(1.1, self.gel.lambda_target, 15, endpoint = False)
        gamma_list = [self.gel.gammafun(la) for la in lambda_list]
        #gamma_list = np.linspace(gamma0, gamma_end, 15, endpoint=False)
        for gamma_i in gamma_list:
            print("*** gamma = ", gamma_i)
            self.gel.gamma.Set(gamma_i)
            self.gfu, _,_ = SolveNonlinearMinProblem(a= self.a, gfu = self.gfu, FreeDofs =self.fes.FreeDofs(), maxits=100, tol=1e-03)
            Draw(self.gfu)

        # final
        print("*** Final gamma = ", gamma_end)
        self.gel.gamma.Set(gamma_end) 
        self.gfu, _,_ = SolveNonlinearMinProblem(a= self.a, gfu = self.gfu, FreeDofs =self.fes.FreeDofs(), maxits=500, tol=1e-06)
        Draw(self.gfu)
    
    def Space(self):
        # Finite element space with slipp boundary conditions on back|bot|left
        self.fes = VectorH1(self.mesh, order=3, dirichlet="bottom")


    def model(self):
        u  = self.fes.TrialFunction()
        I = Id(self.mesh.dim)
        F = I + Grad(u)
        # hydrogel model
        self.a = BilinearForm(self.fes)
        self.a += Variation(  self.gel.W(F).Compile() * dx)
    
    def plot(self):
        Draw(self.gfu)

def SolveNonlinearMinProblem(a, gfu, FreeDofs, tol=1e-08, maxits=50):#, scenes=None):
    res = gfu.vec.CreateVector()
    du  = gfu.vec.CreateVector()

    for it in range(maxits):
        print ("Newton iteration {:3}".format(it),end=", ")
        print ("energy = {:16}".format(a.Energy(gfu.vec)),end="")
        #solve linearized problem:
        a.Apply (gfu.vec, res)
        a.AssembleLinearization (gfu.vec)
        inv = a.mat.Inverse(FreeDofs)
        alpha = 5e-2
        du.data = alpha * inv * res

        #update iteration
        gfu.vec.data -= du

        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break
        #for sc in scenes:
        #    sc.Redraw()
    return gfu, stopcritval, it


In [4]:

GEL = gel_bonded()

modelling = Solve_gel_bonded(GEL, maxh=3, corner_refinement=False)
modelling.Solve_incremental_softening()

#modelling = Solve_gel_bonded(GEL, maxh=2, corner_refinement=True)
#modelling.Solve_incremental_softening()
#modelling.save_to_vtk(filename="result_bonded_sub3_h2_refined")
'''
modelling.Space()
modelling.gfu = GridFunction(modelling.fes)

modelling.model()
lambda_list = [ 1.1, 1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0]# ,1.3,1.4,1.5,1.6,1.7,1.8,1.9]
for lmbd in lambda_list:
    gamma = GEL.gammafun(lmbd)
    GEL.gamma.Set(gamma)
    print("gamma:", GEL.gamma)
    modelling.SolveNonlinearMinProblem(maxits=10)

gamma = GEL.gammafun(2.1)
GEL.gamma.Set(gamma) 
modelling.SolveNonlinearMinProblem(maxits=500)

Draw(modelling.gfu)
#modelling.save_to_vtk()
'''


*** gamma =  0.006429669657915275
Newton iteration   0, energy = -244.6292739508086<A u 0 , A u 0 >_{-1}^0.5 =  0.16315925369855377
Newton iteration   1, energy = -244.655231023697<A u 1 , A u 1 >_{-1}^0.5 =  0.15563377268747983
Newton iteration   2, energy = -244.67884893418108<A u 2 , A u 2 >_{-1}^0.5 =  0.14848028617703762
Newton iteration   3, energy = -244.70034564294792<A u 3 , A u 3 >_{-1}^0.5 =  0.14166363265558632
Newton iteration   4, energy = -244.71991385558226<A u 4 , A u 4 >_{-1}^0.5 =  0.1351579006819832
Newton iteration   5, energy = -244.7377260301771<A u 5 , A u 5 >_{-1}^0.5 =  0.12894294393745026
Newton iteration   6, energy = -244.75393773338638<A u 6 , A u 6 >_{-1}^0.5 =  0.12300234365302472
Newton iteration   7, energy = -244.76869002562603<A u 7 , A u 7 >_{-1}^0.5 =  0.11732217285244087
Newton iteration   8, energy = -244.78211124821968<A u 8 , A u 8 >_{-1}^0.5 =  0.11189022412883533
Newton iteration   9, energy = -244.794318425669<A u 9 , A u 9 >_{-1}^0.5 =  0.1

Newton iteration  81, energy = -244.91374107451378<A u 81 , A u 81 >_{-1}^0.5 =  0.002866631533049126
Newton iteration  82, energy = -244.91374908666404<A u 82 , A u 82 >_{-1}^0.5 =  0.0027235805103194973
Newton iteration  83, energy = -244.91375631911936<A u 83 , A u 83 >_{-1}^0.5 =  0.002587654736150669
Newton iteration  84, energy = -244.91376284768722<A u 84 , A u 84 >_{-1}^0.5 =  0.0024585006027716826
Newton iteration  85, energy = -244.91376874081527<A u 85 , A u 85 >_{-1}^0.5 =  0.002335781925020043
Newton iteration  86, energy = -244.91377406030284<A u 86 , A u 86 >_{-1}^0.5 =  0.0022191790941950415
Newton iteration  87, energy = -244.91377886194567<A u 87 , A u 87 >_{-1}^0.5 =  0.002108388271803936
Newton iteration  88, energy = -244.91378319611957<A u 88 , A u 88 >_{-1}^0.5 =  0.00200312062143817
Newton iteration  89, energy = -244.9137871083045<A u 89 , A u 89 >_{-1}^0.5 =  0.0019031015770846738
Newton iteration  90, energy = -244.9137906395592<A u 90 , A u 90 >_{-1}^0.5 =  

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

*** gamma =  0.005271604593790231
Newton iteration   0, energy = -249.00433307446914<A u 0 , A u 0 >_{-1}^0.5 =  0.09898256231328621
Newton iteration   1, energy = -249.01388613766122<A u 1 , A u 1 >_{-1}^0.5 =  0.09431166110293987
Newton iteration   2, energy = -249.0225588623156<A u 2 , A u 2 >_{-1}^0.5 =  0.08985442141398421
Newton iteration   3, energy = -249.0304311889951<A u 3 , A u 3 >_{-1}^0.5 =  0.08560063592754917
Newton iteration   4, energy = -249.03757578329274<A u 4 , A u 4 >_{-1}^0.5 =  0.08154089243784704
Newton iteration   5, energy = -249.04405875114892<A u 5 , A u 5 >_{-1}^0.5 =  0.07766642219661149
Newton iteration   6, energy = -249.04994026178824<A u 6 , A u 6 >_{-1}^0.5 =  0.07396899050124614
Newton iteration   7, energy = -249.05527509696066<A u 7 , A u 7 >_{-1}^0.5 =  0.07044081670143133
Newton iteration   8, energy = -249.06011313994344<A u 8 , A u 8 >_{-1}^0.5 =  0.06707451492542732
Newton iteration   9, energy = -249.06449981413544<A u 9 , A u 9 >_{-1}^0.5 =

Newton iteration  81, energy = -249.1064225643774<A u 81 , A u 81 >_{-1}^0.5 =  0.001661676018905523
Newton iteration  82, energy = -249.10642525651767<A u 82 , A u 82 >_{-1}^0.5 =  0.00157868560387149
Newton iteration  83, energy = -249.1064276864626<A u 83 , A u 83 >_{-1}^0.5 =  0.0014998356171113674
Newton iteration  84, energy = -249.1064298797334<A u 84 , A u 84 >_{-1}^0.5 =  0.0014249199219211388
Newton iteration  85, energy = -249.1064318593715<A u 85 , A u 85 >_{-1}^0.5 =  0.0013537426024309513
Newton iteration  86, energy = -249.10643364617667<A u 86 , A u 86 >_{-1}^0.5 =  0.001286117460911601
Newton iteration  87, energy = -249.10643525892348<A u 87 , A u 87 >_{-1}^0.5 =  0.0012218675394071548
Newton iteration  88, energy = -249.10643671456083<A u 88 , A u 88 >_{-1}^0.5 =  0.0011608246645552689
Newton iteration  89, energy = -249.10643802838771<A u 89 , A u 89 >_{-1}^0.5 =  0.0011028290145061357
Newton iteration  90, energy = -249.1064392142146<A u 90 , A u 90 >_{-1}^0.5 =  0

...

*** Final gamma =  0.0009516837481698391
Newton iteration   0, energy = -268.829283037408<A u 0 , A u 0 >_{-1}^0.5 =  0.03662636107470206
Newton iteration   1, energy = -268.8305909655333<A u 1 , A u 1 >_{-1}^0.5 =  0.03475443244672318
Newton iteration   2, energy = -268.83176861837296<A u 2 , A u 2 >_{-1}^0.5 =  0.03298072463842947
Newton iteration   3, energy = -268.83282913573015<A u 3 , A u 3 >_{-1}^0.5 =  0.03129979621665644
Newton iteration   4, energy = -268.833784306208<A u 4 , A u 4 >_{-1}^0.5 =  0.0297065385158023
Newton iteration   5, energy = -268.8346447101973<A u 5 , A u 5 >_{-1}^0.5 =  0.028196152145748533
Newton iteration   6, energy = -268.8354198471158<A u 6 , A u 6 >_{-1}^0.5 =  0.026764125429031053
Newton iteration   7, energy = -268.8361182487254<A u 7 , A u 7 >_{-1}^0.5 =  0.025406214590639777
Newton iteration   8, energy = -268.8367475801293<A u 8 , A u 8 >_{-1}^0.5 =  0.024118425540834185
Newton iteration   9, energy = -268.8373147298497<A u 9 , A u 9 >_{-1}^0.5

Newton iteration  80, energy = -268.8425154414852<A u 80 , A u 80 >_{-1}^0.5 =  0.0005939092730056591
Newton iteration  81, energy = -268.8425157853953<A u 81 , A u 81 >_{-1}^0.5 =  0.0005642080065155624
Newton iteration  82, energy = -268.84251609576785<A u 82 , A u 82 >_{-1}^0.5 =  0.0005359923745035519
Newton iteration  83, energy = -268.8425163758734<A u 83 , A u 83 >_{-1}^0.5 =  0.0005091880387941236
Newton iteration  84, energy = -268.84251662866416<A u 84 , A u 84 >_{-1}^0.5 =  0.0004837243837325422
Newton iteration  85, energy = -268.84251685680374<A u 85 , A u 85 >_{-1}^0.5 =  0.00045953432949742814
Newton iteration  86, energy = -268.84251706269646<A u 86 , A u 86 >_{-1}^0.5 =  0.0004365541548027884
Newton iteration  87, energy = -268.8425172485113<A u 87 , A u 87 >_{-1}^0.5 =  0.0004147233285165912
Newton iteration  88, energy = -268.84251741620636<A u 88 , A u 88 >_{-1}^0.5 =  0.0003939843497447425
Newton iteration  89, energy = -268.84251756754963<A u 89 , A u 89 >_{-1}^0.

Newton iteration 159, energy = -268.8425189672451<A u 159 , A u 159 >_{-1}^0.5 =  1.0322811583495379e-05
Newton iteration 160, energy = -268.842518967349<A u 160 , A u 160 >_{-1}^0.5 =  9.806669286056598e-06
Newton iteration 161, energy = -268.84251896744223<A u 161 , A u 161 >_{-1}^0.5 =  9.316334271029556e-06
Newton iteration 162, energy = -268.8425189675271<A u 162 , A u 162 >_{-1}^0.5 =  8.850516158073898e-06
Newton iteration 163, energy = -268.84251896760355<A u 163 , A u 163 >_{-1}^0.5 =  8.407989087025435e-06
Newton iteration 164, energy = -268.84251896767233<A u 164 , A u 164 >_{-1}^0.5 =  7.987588492807985e-06
Newton iteration 165, energy = -268.84251896773463<A u 165 , A u 165 >_{-1}^0.5 =  7.588208039672232e-06
Newton iteration 166, energy = -268.84251896779125<A u 166 , A u 166 >_{-1}^0.5 =  7.20879670927112e-06
Newton iteration 167, energy = -268.8425189678419<A u 167 , A u 167 >_{-1}^0.5 =  6.848356036192753e-06
Newton iteration 168, energy = -268.84251896788777<A u 168 ,

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

'\nmodelling.Space()\nmodelling.gfu = GridFunction(modelling.fes)\n\nmodelling.model()\nlambda_list = [ 1.1, 1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0]# ,1.3,1.4,1.5,1.6,1.7,1.8,1.9]\nfor lmbd in lambda_list:\n    gamma = GEL.gammafun(lmbd)\n    GEL.gamma.Set(gamma)\n    print("gamma:", GEL.gamma)\n    modelling.SolveNonlinearMinProblem(maxits=10)\n\ngamma = GEL.gammafun(2.1)\nGEL.gamma.Set(gamma) \nmodelling.SolveNonlinearMinProblem(maxits=500)\n\nDraw(modelling.gfu)\n#modelling.save_to_vtk()\n'